In [ ]:
# SETTING UP OPENAI
# Import necessary libraries and modules
from dotenv import load_dotenv
import os
import re
from enum import Enum
import openai

# Load environment variables from a specified .env file for secure API key storage
env_path = '/home/msd4/aidocsMosi/openAI_Token.env'
load_dotenv(dotenv_path=env_path)

# Retrieve the OpenAI API key from the environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    # Raise an error if the API key is not found, to prevent API calls without authentication
    raise ValueError("No API key found. Please set your OPENAI_API_KEY in the .env file.")

# Initialize the OpenAI client with the retrieved API key
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Define supported OpenAI models and their token limits as an Enum for better code readability and maintainability
# For model selection and token lengths see: https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo
class OpenAIModels(Enum):
    GPT3_TURBO = "gpt-3.5-turbo"
    GPT3_TURBO_UPDATED = "gpt-3.5-turbo-0125" 
    GPT4 = "gpt-4"
    GPT4_TURBO = "gpt-4-0125-preview"

# Map models to their ca. 1/3 or smaller than the respective token limits
MODEL_TOKEN_LIMITS = {
    OpenAIModels.GPT3_TURBO: 8000,
    OpenAIModels.GPT3_TURBO_UPDATED: 3000,
    OpenAIModels.GPT4: 3000,
    OpenAIModels.GPT4_TURBO: 8000,  
}

In [ ]:
def expand_dialogue(input):
    input_dialogue = input #input is the dialogue to be expanded

    #------------------------------------------------------------------------------------------------------------
    # CREATE GERMAN OUTLINES FROM ENGLISH INPUT-DIALOGUES INTO outline abschnitte = []
    #------------------------------------------------------------------------------------------------------------
    
    outline_abschnitte = [] #Global list to store outlines of each 'Abschnitt'

    '''
    def split_query_into_parts(query: str, limit: int) -> list[str]:
        """Split a large text query into manageable parts according to the token limit."""
        return concatenate_sentences(split_into_sentences(query), limit)


    def split_into_sentences(text: str) -> list[str]:
        pattern = r"(?<=[.!?])\s+"
        return re.split(pattern, text)


    def concatenate_sentences(sentences: list[str], limit: int) -> list[str]:
        concatenated, temp, total_length = [], [], 0
        for sentence in sentences:
            temp.append(sentence)
            total_length += len(sentence) + 1  # Accounting for space between sentences
            if total_length >= limit:
                concatenated.append(" ".join(temp))
                temp, total_length = [], 0
        # Add any remaining sentences as the last chunk
        if temp:
            concatenated.append(" ".join(temp))
        return concatenated
    '''   

    def send_request(query: str, model: OpenAIModels) -> str:
        print("Jetzt läuft erster %def send_request%: der englischer input_dialogue nimmt und summarized und den abschnitt zuweist")
        summarize_instruction = f"""Deine Aufgabe ist es einen Gesprächsverlauf während eines Rettungsdiensteinsatzes 
        bis zur Anmeldung in die Klinik auf Deutsch zu beschreiben: 
        Input-Dialog: {query}
        Achte dabei, dass du den Gesprächsverlauf zusammmefassend wiedergibts und nach Inhalten und Themen in Abschnitten aufteilst, 
        indem du alle medizinischen Inhalte oder durchgeführte Massnahmen mitteinschliesst, 
        wie sie in Deutschen oder Schweizerischen etablierten Rettungswesen gängig sind. 
        Versuche dabei zu erkennen ob die Inhalte des Input-Dialog in welche der folgenden  Kapiteln zuzuteilen, bei denen der Inhalt passt: 
        1. Administrative Metadaten & Einsatzstellenorganisation:
         - Adresse(n), Namen, Beteiligte, Hinweise, Gefährdungspotential(e), Lage, Beteiligte involvierte Patienten/Personen, 
         - Topografie, 
         - Unfallmechanismen
         - Patientenpositionen
         - Zugang zur Unfall- oder Notfallstelle
         - Namen der involvierten medizinischen Fachpersonen, (falls keine realisitische Namen vorhanden erfinde hier welche).
        3. ABCDE Scheme Screening o. Vorstellung Team vs. Patient:
         - A Atemweg; B Beatmung;  C Kreislauf; D Defizit, neurologisches; E - Exposure/Environment (Exploration)
        4. Anamneserhebung:
         - Patientengespräch oder durch Angehörige oder externe Personen
        5. Anschluss Geräte & Untersuchung des Patienten / Diagnostik (Vom Kopf zu den Füssen):
         - Sauerstoffinsufflaton
         - SpO2
         - EKG
         - Blutdruck
         - Überwachung des Patienten
         - Blutungen
         - Verletzungen
         - Prellmarken
         - Hautzustand
         - Beweglichkeiten
         - Hörbares
         - Riechbares
         - Druckdolenzen
         - Haltungen
         - Scores erheben (GCS)
         - Sensorik beurteilen
         - Arbeitsdiagnose gewinnen
         - Überwachung des Patienten
        7. Massnahmenplanung:
         - Lagerungsmassnahmen
         - Beatmung
         - Herzdruckmassage
         - Gefässzugang pripher
         - Gefässzugang zentral
         - Intraossärer Zugang
         - Flüssigkeitstherapie
         - Medikamentöse Therapie
         - Narkoseeinleitung
         - Atemwegszugang
         - Thoraxpunktion
         - Blutungsstillung
         - HWS-Immobilisation
         - Wirbelsäulen- Immobilisation
         - Repositionen
         - Extremitäten- Immobilisierungen
         - Überwachung des Patienten
        8. Massnahmendurchführung & Massnahmen- Ergebniskontrolle
         - Lagerungsmassnahmen
         - Beatmung
         - Herzdruckmassage
         - Gefässzugang pripher/zentral
         - Intraossärer Zugang
         - Flüssigkeitstherapie
         - Medikamentöse Therapie
         - Narkoseeinleitung
         - Atemwegszugang
         - Thoraxpunktion
         - Blutungsstillung
         - HWS-Immobilisation
         - Wirbelsäulen- Immobilisation
         - Repositionen
         - Extremitäten- Immobilisierungen
         - Überwachung des Patienten
         - Auswertung Sensorik
         - Nachuntersuchungen
        10. Entscheidung Zielklinik & Transportfähigkeit herstellen
         - angepasst an das Krankheits- oder Verletzungsausmass
         - Maximal-, Regel- oder Basisversorgung
         - Psychiatrische Einrichtung
         - Auswahl Rettungsmittel
         - Eigenes Rettungsmittel
         - Helikopter
         - anderes Fahrzeug
         - Zustand des Patienten
         - transportfähig oder nicht transportfähig
        12. Transport in das Rettungsmittel
         - Transportmittel; Trage; Stuhl; Bergungstuch; Vakuummatratze; Schaufeltrage
        13. Kontrolle im Rettungsmittel
         - Zustand des Patienten
         - transportfähig / nicht transportfähig
         - Entscheidung weitere Massnahmen
         - Überwachung des Patienten
        14. Massnahmen im Rettungsmittel
         - Massnahmen zur Herstellung oder Haltens der Transportfähigkeite
         - Massnahmen, die ausserhalb des Rettungsmittels problematisch waren
         - Narkoseeinleitung
         - Atemweg
         - Zugänge
         - Überwachung des Patienten
        15. Anmeldung Klinik im Rettungsmittel & Transport in die Klinik & Transport in die Klinik & Nachbesprechung
         - Patientangaben gemäss Datenschutzverordnug
         - Verletzungs- oder Erkrankungsmuster
         - Kurzüberblick Massnahmen
         - Zustand des Patienten
         - ETA
         - Entscheidung Sondersignal
         - Überwachung des Patienten
         - An den Zustand des Patienten angepasster Transport
        und füge die Dialoginhalte in die passenden Schema-Abschnitte ein, 
        dort wo du keine Inhalte zuweisen kannst versuche aus dem Input-Dialog realistische Inhalte zu erfinden an dem zu dich an den Unterpunkten
        der jeweiligen Abschnitten Orientierts."
        
        Bitte achte unbedingt darauf, dass du aus dem "Input-Dialog"
        folgendes genau so exakt wiedergibts und immer in deiner Antwort integrierst: 
        -alle vorhandenen medizinischen Messwerte, Vitalwerte, Atemzüge oder Respirationen pro Minute, Schmerzskala-Werte, Schmerzbeschreibungen, Schmerzscores, numerische Schmerzangaben (pain scores),
        - die erwähnten Interventionen, Massnahmen und Behandlungen,
        - Medikamentennamen, Medikamentenwirkstoffe und Medikamentendosierungen, 
        - medizinische oder unfalltechnische Vorgeschichte, Diagnosen, vermutete und gestellte Diagnose, Symptome und gesundheitichen Beschwerden oder Informationen.
        
        Besonders bei den Medikamentennamen, Gesundheitsprobleme, medizinische Vorgeschichte und Diagnosen, musst du genau diese wiedergeben.
        
        Achte unbedingt darauf, dass du deine Ergebnisse pro Abschnitte immer in diesem Format ausgibts!: 
        Abschnitt Titel: <titel> 
        Geschehnisse: <geschehnisse>. 
        '\n\n'
        """ 
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": summarize_instruction}], temperature=0.1,
            model=model.value,
        )
        # Access the content properly based on the response object's structure
        return response.choices[0].message.content  
    
    def handle_request_and_collect_summaries(input_dialogue: str, model: OpenAIModels) -> list[str]:
        """Process input dialogue by summarizing under thematic outlines and separate each outline to save them into a list of outlines."""
        token_limit = MODEL_TOKEN_LIMITS[model] 
        #parts = split_query_into_parts(input_dialogue, token_limit)
        input_dialogue = input_dialogue

        summaries = []  # Initialize an empty list to collect summaries

      
        summarized_dialogue = send_request(input_dialogue, model)
        print("summarized_dialogue as a whole")
        print(summarized_dialogue)
        
        
        abschnitte = summarized_dialogue.split('\n\n')  # Assuming each 'Abschnitt' is separated by two newlines
        for abschnitt in abschnitte:
            print("a single outline has been appended to the list of outlines.")
            if abschnitt.strip():  # Making sure it's not an empty string
                outline_abschnitte.append(abschnitt.strip())
            
        print("\n--- Ende der Outline herstellung: ---\n")
        
        return summaries  # Return the list of summaries for further processing


    """    
        for part in parts:
            summarized_chunk = send_request(part, model)
            print("print(summarized_chunk) sollte ein  zusammengefasster chunk auf deutsch mit  Abschnitt Titel: <titel> und Geschehnisse: <geschehnisse>. sein: ")
            print(summarized_chunk)
            summaries.append(summarized_chunk)  # Append the summarized part to the list
        
            # Split the summarized part into individual 'Abschnitte' and add them to the global list
            abschnitte = summarized_chunk.split('\n\n')  # Assuming each 'Abschnitt' is separated by two newlines
            for abschnitt in abschnitte:
                if abschnitt.strip():  # Making sure it's not an empty string
                    outline_abschnitte.append(abschnitt.strip())
            print("\n--- Ende der Zusammenfassung des Chunks ---\n")
        
        return summaries  # Return the list of summaries for further processing
    """       
        
    def main():
        """Main function to execute the summarization process and capture summaries."""
        model = OpenAIModels.GPT4_TURBO  # Model for summarizing the input dialogue for outline-generation
        #summaries = handle_request_and_collect_summaries(input_dialogue, model)  # Begin the summarization process and collect summaries
        handle_request_and_collect_summaries(input_dialogue, model)  # Begin the summarization process and collect summaries
    
    # Execute the main function
    main()

    #------------------------------------------------------------------------------------------------------------
    # FROM HERE ON: EXPANDING DIALOGUES *
    #------------------------------------------------------------------------------------------------------------
    
    def generate_detailed_dialogue_for_abschnitt(abschnitt: str, summary_for_context: str, model, temperature: float, max_tokens: int) -> str:
        """Generates detailed expanded Dialogues per "Abschnitt"."""
        # Extrahiere den Titel und die Ereignisse aus dem 'Abschnitt'
        title_events_split = abschnitt.split('\nGeschehnisse: ')
        title_part = title_events_split[0]
        events = title_events_split[1] if len(title_events_split) > 1 else ""
        title = title_part.replace('Abschnitt Titel: ', '')
        
        # Konstruiere den Prompt für die Generierung des detaillierten Dialogs
        detailed_dialogue_prompt = (
            f"Zusammenfassung des letzten Dialogs: {summary_for_context}\n\n" 
            if summary_for_context else "Keine Zusammenfassung vorhanden, da Beginn des Dialogs."
        ) + (
            f"Abschnitt Titel: {title}\n"
            f"Aktuelle Geschehnisse: {events}\n\n"
    
            f"""Entwickle einen detaillierten und realistischen Dialog, der nur die gesprochenen Worte der Personen umfasst.
            Stelle sicher, dass die Dialoge authentisch sind und der Situation während eines Rettungsdiensteinsatzes im Deutschland
            angemessen sind und auf den 'Aktuellen Geschehnissen' basieren.
            Die medizinischen Fachkräfte sollen untereinander fachspezifisch kommunizieren, 
            während sie mit dem Patienten in allgemeinverständlicher medizinischer und höflicher Sprache sprechen.
            Passe die Antworten des Patienten seinem medizinischen Zustand an, z.B. wenn er verwirrt ist 
            oder sich in einem kritischen Zustand befindet, so soll er dementsprechend kommunizieren. 
            Integriere Variabilität in die Dialoge, um die natürliche Art der Konversation widerzuspiegeln. 
            Achte unbedingt darauf dass du die unter 'Aktuelle Geschehnisse:' erwähnten Diagnosen, Symptomen, Gesundheitsbeschwerden, Medikamente, Medikamentennamen,
            Medikamentenwirkstoffe, Dosierungen ,medizinische Maßnahmen und Messwerte genau und vollständig in die Dialoge integrierst.
            Vermeide es, generelle Informationen zu wiederholen, die bereits erwähnt wurden, und konzentriere dich darauf, 
            den Dialog natürlich basierend auf den 'Zusammenfassung des letzten Dialogs' weiterzuführen. Achte darauf die neuen Dialoge auf die 
            medizinischen Maßnahmen anzupassen, z.B. sollen die medizinischen Fachpersonen untereinander diese klar formulieren 
            und wenn angewendet am Patienten diesen miteinbeziehen. Übernehme stets bereits erwähnten Namen.
            Beachte folgendes:
            Im Abschnitt: 
            "1. Administrative Metadaten & Einsatzstellenorganisation:" finden nur Gespräche unter dem Rettungspersonal statt die 
            einen Bezug auf die Geschehnisse unter Abschnitt: "1. Administrative Metadaten & Einsatzstellenorganisation:" haben.
            Nur im im Abschnitt: "3. ABCDE Scheme Screening & Vorstellung Team vs. Patient:" sollen sich die medizininschen Fachpersonen 
            dem Patienten vorstellen, falls der Patient bei Bewusstsein ist.  
            Nur im Abschnitt: "5. Anschluss Geräte & Untersuchung des Patienten" sollen die Fachpersonen den Patienten informieren, 
            dass er für einen IV-Zugang gestochen wird und ihm Messgeräte zur Messung der Vitalzeichen angelegt werden. 
            Halte den Dialog innerhalb des Kontexts der neuen Ereignisse und vermeide Beschreibungen sowie Satzzeichen oder Punktuation 
            und vermeide Sätze wie "Halte durch" oder "Wir sind bald da" die sich nicht auf medizinische Massnahmen beziehen 
            und vermeide Wiederholungen.
            
            Achte unbedingt darauf, dass du aus 'Aktuelle Geschehnisse:' folgende Informationen exakt, genau und vollständig in die Dialoge einbaust:
            Alle medizinischen Messwerte und Resultate ohne Umrechnung, alle medizinische Informationen wie Symptome, Diagnosen, Verdachtsdiagnosen, Medikamente, Angaben zu Schmerzen wie Schmwerzscores,
            Medikamentenwirkstoffe, Medikamentendosierungen!!!
        
            """
        )
        
        print("*******************************************************")
        print("detailed_dialogue_prompt:  \n")
        print(detailed_dialogue_prompt)
        print("-------------------------------")
        
        # Sende die Anfrage an die OpenAI API
        detailed_dialogue_response = send_request(detailed_dialogue_prompt, model, temperature, max_tokens)
        
        print("detailed_dialogue_response:  \n")
        print(detailed_dialogue_response)
        print("*******************************************************")
    
        return detailed_dialogue_response


    def send_request(prompt: str, model: OpenAIModels, temperature: float, max_tokens: int) -> str:
        """Send a request to the OpenAI API with a given prompt, model, and temperature."""
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            model=model.value,
            max_tokens=max_tokens  # Specify the maximum length of the output
        )
        # Access the content properly based on the response object's structure
        print("HIER DER ZWEIZE send_request im generate dialogue response)")
        return response.choices[0].message.content

    
    def generate_summary(text: str, model: OpenAIModels, token_limit: int) -> str:
        """
        Generate a summary for a given text. This is a simplified version.
        """
        prompt = f"""Mache eine Zusammenfassung was bereits gesagt wurde und was bereits geschehen ist: {text} und erwähne 
        alle Messgeräte für Vitalzeichen die angelegt wurden, die IV-Zugänge die gelegt wurden, 
        die Vitalwerte und die Namen der involvierten Personen und ob der Patient ansprechbar sein kann aufgrund seines Zustandes."""
        response = client.chat.completions.create(
            model=model.value,
            messages=[{"role": "system", "content": prompt}],
            temperature=0.5,
            max_tokens=token_limit
        )
        return response.choices[0].message.content if response.choices else "Summary generation failed."
    
    
    # Adjust the parameters as desired
    temperature_setting = 0.1
    max_output_length = 4000 
    model = OpenAIModels.GPT4
    
    final_dialogue = ""
    summary_for_context = ""
    last_dialogue =""

    print("outline_abschnitte:____________")
    print(outline_abschnitte)
    

    for i, abschnitt_str in enumerate(outline_abschnitte):
        # Splitting the string into parts based on the known labels in the string
        parts = abschnitt_str.split(' Geschehnisse: ')
        print("parts:________parts = abschnitt_str.split(' Geschehnisse: ')__________")
        print(parts)
        title = parts[0].replace('Abschnitt Titel: ', '').strip()
        events = parts[1] if len(parts) > 1 else ""
    
        if i > 0:
            # Generate a summary of final_dialogue for context
            summary_for_context = generate_summary(detailed_dialogue, model, max_output_length)
    
        # Generate detailed dialogue for the current section with context, using the modified function signature
        detailed_dialogue = generate_detailed_dialogue_for_abschnitt(abschnitt_str, summary_for_context, model, temperature_setting, max_output_length)

        # Append the generated dialogue to final_dialogue
        final_dialogue += detailed_dialogue + "\n"

        
        # Optionally print or log the intermediate results for debugging
        print(f"Processed section {i+1}/{len(outline_abschnitte)}")
        

    return final_dialogue

In [ ]:
from pymongo import MongoClient

# Assuming you have MongoDB running locally. Replace the URI with your connection details
mongod_client = MongoClient('mongodb://localhost:27017/')
db = mongod_client['MIMIC-IV']

# Access the collection containing the dialogues
dialogues_collection = db['NLP-EVAL-prammed']  # Source Collection


# ---------------Process Sample from csv import---------------------------

This Section was used for prompttesting after first validation of RAG turned out bad.

1. Load and merge the _id values from the CSV files.

2. Fetch the corresponding stay_id for these _id values from the NLP-EXPANDED-prammed collection.

3. Use the stay_id to fetch the right entry from the NLP-EVAL-prammed collection.

4. Process the dialogues and save the results into NLP-EXP-pram-sampled.

## Here we do sampling, test

# ----------------ENDE TEST OF SAMPLES ----------------------

## Processing via loop through each Doc in Source Collection

This is the simplest way to process. Best if target collection is empty.

In [ ]:
# A Loop that loops trough mongodb-collection "NLP-EVAL-prammed" and creates an expanded dialogue for each item's "dialogue" and saves an expanded "final_dialogue" in the collection NLP-EXPANDED-prammed.
for entry in dialogues_collection.find():
    dialogue = entry.get('dialogue', '')
    final_dialogue = expand_dialogue(dialogue)  # Make sure this returns a valid string, not None
    
    # Explicitly define the new document to insert
    expanded_entry = {
        **entry,  # This copies existing fields from the original document
        'final_dialogue': final_dialogue  # Ensures final_dialogue is explicitly set
    }
    
    # Remove _id to avoid duplicate key error
    expanded_entry.pop('_id', None)
     
    db['NLP-EXPANDED-prammed-2'].insert_one(expanded_entry) # Target Collection


# Other Processing Ways/Methods/Functions

#### Process single dialogue by single "_id" Object-Id in mongo-collection

#### Ranged processing